In [1]:
from pea.mail_util import Emails

email_obj = Emails()

In [8]:
# Remove tool decorator to run
# email_obj.get_mails_content(from_addr="noreply@redditmail.com")

In [9]:
# Remove tool decorator to run
# email_obj.get_mails_content(filter="SEEN", from_addr="noreply@redditmail.com")

In [2]:
print(email_obj.get_mails_content.args)

{'filter': {'description': 'Filter on mails supported by IMAPClient like UNSEEN', 'title': 'Filter', 'type': 'string'}, 'from_addr': {'description': "Sender's email address", 'title': 'From Addr', 'type': 'string'}}


In [3]:
from langgraph.prebuilt import ToolNode
from pea.mail_util import get_mails_content_tool

tools = [get_mails_content_tool]

tool_node = ToolNode(tools)

In [4]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="mistral", temperature=0).bind_tools(tools)

In [5]:
from langgraph.graph import END, StateGraph, MessagesState
from typing import Annotated, Literal, TypedDict

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    response = llm.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


In [6]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", 'agent')

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile(checkpointer=checkpointer)


In [7]:
#  Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="unseen mails from noreply@redditmail.com")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content


" It seems that the function `get_mails_content` is not defined correctly. The error message suggests that the function is missing a required positional argument, which should be `self`. Here's an example of how you can define a class method with the correct syntax:\n\n```python\nclass EmailHandler:\n    def get_mails_content(self):\n        # Your code here to fetch and return mail content\n\n# Then you can call this method on an instance of the EmailHandler class like so:\nemail_handler = EmailHandler()\nemail_handler.get_mails_content()\n```"